# Transformación de Datos 
****

***Se importan librerias necesarias***

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast 
import re
from sklearn.preprocessing import MultiLabelBinarizer

# Transformación de datos: Dataset Steam_games
***

**Se carga y se convierten en DataFrame para su visualización correcta**

In [ ]:
steam_games = pd.read_json('/Users/carolina/Desktop/Data/Henry - Data Science/M7/Proyecto Individual 1/steam_games.json',lines=True)
df1 = steam_games
df1

**Se realiza exploracion de los datos**

In [ ]:
df1.info()
df1.describe()
df1.shape

**Se revisan y se eliminan filas con datos nulos en el DataFrame**


In [ ]:
df1.isnull().sum()

In [4]:
df1 = df1.dropna()

**# Se convierten las columnas con listas a cadenas de texto para poder buscar duplicados**

In [6]:
# Convertir las columnas con listas a cadenas de texto para poder buscar duplicados
df1 = df1.applymap(lambda x: tuple(x) if isinstance(x, list) else x)

# Encontrar los duplicados
duplicados = df1[df1.duplicated()]

# Mostrar los duplicados
print("Duplicados encontrados:")
print(duplicados)

/var/folders/mz/q_bk1n_55k70r74tttm3ps0c0000gn/T/ipykernel_7277/3296340217.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x: tuple(x) if isinstance(x, list) else x)


Duplicados encontrados:
Empty DataFrame
Columns: [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Index: []


**Eliminar filas con valores NaN en cualquier columna y se verifica el df**

In [ ]:
df1 = df1.dropna()
print(df1.head())

**Transformar la columna price (tipo de dato, convertir los valores no numericos en NaN y despues rellenar con 0)**

In [8]:
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')  # Se convierte valores no numéricos en NaN
df1['price'] = df1['price'].fillna(0)  # Se rellena los NaN con 0 
df1['price'] = df1['price'].astype(int)  # Se convierte a tipo entero

**Se cambia de tipo de dato la columna 'release_date'**

In [9]:
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce')

**Se convierte todas las columnas de tipo 'object' a 'str'**

In [10]:
for col in df1.select_dtypes(include=['object']).columns:
    df1[col] = df1[col].astype(str)
    df1['id'] = df1['id'].astype(str)

***Se exporta el archivo para el sistema de recomendación***

In [45]:
df1.to_parquet('df1_games.parquet', engine='pyarrow', index=False)

In [ ]:

columnas_especificas = ['publisher','genres', 'title', 'release_date','price','id','developer']

# Crear un nuevo DataFrame con solo las columnas específicas
nuevo_df_games = df1[columnas_especificas]

# Ver el nuevo DataFrame
print(nuevo_df_games)
df_games = nuevo_df_games

***Se exporta el archivo para la API***

In [111]:
df_games.to_parquet('df_games.parquet', engine='pyarrow', index=False)

# Transformacion: Dataset user_reviews

**Se carga y se convierten en DataFrame para su visualización correcta**

In [48]:
list_reviews = [] #Este código está diseñado para leer un archivo JSON línea por línea, convertir cada línea en un diccionario Python y luego agregarlo a una lista llamada list_reviews
archivo3 = r'/Users/carolina/Desktop/Data/Henry - Data Science/M7/Proyecto Individual 1/user_reviews.json'
with open(archivo3, encoding='utf-8') as file2:
    for line2 in file2.readlines():
        list_reviews.append(ast.literal_eval(line2))
it2= pd.DataFrame(list_reviews)
df3 = it2

***Desanidar Columna "reviews" del Dataframe***

In [ ]:

#Este codigo funciona para limpiar la columna "reviews", convirtiendo cualquier texto que represente una lista o diccionario en una estructura de Python real.
df3['reviews'] = df3['reviews'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Verificar si hay índices duplicados
duplicate_index = df3.index.duplicated()
if duplicate_index.any():
    print(df3[duplicate_index])  # No se encontraron indices duplicados
# Restablecer el índice
df3_reset = df3.reset_index(drop=True)
# Convertir la columna 'reviews' en listas de diccionarios
df3['reviews'] = df3['reviews'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Explota las reseñas
df_exploded = df3.explode('reviews')

# Normaliza la columna 'reviews'
df_reviews = pd.json_normalize(df_exploded['reviews'])

# Restablece el índice del DataFrame normalizado
df_reviews.reset_index(drop=True, inplace=True)

# Combina los DataFrames
df_final_reviews = pd.concat([df_exploded.drop(columns=['reviews']).reset_index(drop=True), df_reviews], axis=1)

# Muestra el DataFrame final
print(df_final_reviews.head())
df_final_reviews
df3= df_final_reviews


In [ ]:
df3

**Se realiza exploracion de los datos**

In [ ]:
df3.info()
df3.describe()
df3.shape

**Se revisan y se eliminan filas con datos nulos en el DataFrame**

In [54]:
df3.isnull().sum()
df3 = df3.dropna()

***Se convierte las columnas con listas a cadenas de texto para poder buscar duplicados**

In [ ]:

df3 = df3.applymap(lambda x: tuple(x) if isinstance(x, list) else x)
# Encontrar los duplicados
duplicados = df3[df3.duplicated()]
# Mostrar los duplicados
print("Duplicados encontrados:")
print(duplicados)
df3 = df3.drop_duplicates()
# Encontrar los duplicados
duplicados = df3[df3.duplicated()]



***Limpiar los datos en la columna review del DS***

In [ ]:

# Función para limpiar texto
def clean_review(text):
    # Eliminar URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    # Eliminar caracteres no alfanuméricos excepto espacios
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convertir el texto a minúsculas
    text = text.lower()
    return text

# Aplicar la limpieza a la columna 'review'
df3['review'] = df3['review'].apply(lambda x: clean_review(x) if isinstance(x, str) else '')

# Eliminar filas donde 'review' esté vacío o no válido
df3 = df3[df3['review'].str.strip() != '']

# Eliminar cualquier reseña vacía o NaN
df3 = df3.dropna(subset=['review'])

print(df3.head())


In [ ]:
df3

***Cambiar los tipos de datos**

In [60]:
df3['item_id'] = df3['item_id'].astype(str)

***Se exporta el archivo para el sistema de recomendación***

In [61]:
df3.to_parquet('df3_reviews.parquet', engine='pyarrow', index=False)

***Se escogen las columnas que se necesitan para realizar las funciones de la API***

In [ ]:

columnas_especificas = ['user_id','item_id','posted','recommend', 'review']

# Crear un nuevo DataFrame con solo las columnas específicas
nuevo_df_items = df3[columnas_especificas]

# Ver el nuevo DataFrame
print(nuevo_df_items)
df_reviews2 = nuevo_df_items

***Se exporta el archivo para Análisis de sentimiento***

In [26]:
df_reviews2.to_parquet('df_reviews2.parquet', engine='pyarrow', index=False)

**Se exporta para la API**

In [124]:
df_reviews.to_parquet('df_reviews.parquet', engine='pyarrow', index=False)

# Transformación: Dataset  user_item


**Se carga y se convierten en DataFrame para su visualización correcta**

In [25]:
list_items = []  #Este código está diseñado para leer un archivo JSON línea por línea, convertir cada línea en un diccionario Python y luego agregarlo a una lista llamada list_items
archivo2 = r'/Users/carolina/Desktop/Data/Henry - Data Science/M7/Proyecto Individual 1/users_items.json'
with open(archivo2, encoding='utf-8') as file2:
    for line2 in file2.readlines():
        list_items.append(ast.literal_eval(line2))

it1 = pd.DataFrame(list_items)
df2 = it1

In [ ]:
df2

***Desanidar Columna "items" del Dataframe***

In [ ]:

# Convertir la columna 'items' en listas de diccionarios en el DataFrame original
df2['items'] = df2['items'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Explota las reseñas en el mismo DataFrame original 
df2 = df2.explode('items')

# Normaliza la columna 'items' en el DataFrame original
df_items = pd.json_normalize(df2['items'])

# Restablece el índice del DataFrame original
df_items.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

# Combina las columnas normalizadas con el DataFrame original
df2 = pd.concat([df2.drop(columns=['items']), df_items], axis=1)

# Muestra el DataFrame final
print(df2.head())



**Se realiza exploración de los datos**

In [28]:
df2.info()
df2.describe()
df2.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


**Se revisan y se eliminan filas con datos nulos en el DataFrame**

In [ ]:
df2.isnull().sum()
df2 = df2.dropna()

**Se verifican y se eliminan duplicados**

In [ ]:
duplicados = df2[df2.duplicated()]
print("Duplicados encontrados:")
print(df2)

In [31]:
# Elimina los duplicados, conservando solo la primera aparición
df2_sin_duplicados = df2.drop_duplicates()

# Verifica el tamaño del DataFrame después de eliminar los duplicados
print(f"Tamaño del DataFrame después de eliminar duplicados: {df2_sin_duplicados.shape}")

Tamaño del DataFrame después de eliminar duplicados: (5110819, 8)


In [32]:
df2_items =df2_sin_duplicados

In [33]:
# Eliminar filas con valores NaN en cualquier columna
df2_items = df2_items.dropna()

# Mostrar el DataFrame después de eliminar filas con NaN
print(df2_items.head())

             user_id  items_count           steam_id  \
0  76561197970982479          277  76561197970982479   
1  76561197970982479          277  76561197970982479   
2  76561197970982479          277  76561197970982479   
3  76561197970982479          277  76561197970982479   
4  76561197970982479          277  76561197970982479   

                                            user_url item_id  \
0  http://steamcommunity.com/profiles/76561197970...      10   
1  http://steamcommunity.com/profiles/76561197970...      20   
2  http://steamcommunity.com/profiles/76561197970...      30   
3  http://steamcommunity.com/profiles/76561197970...      40   
4  http://steamcommunity.com/profiles/76561197970...      50   

                   item_name  playtime_forever  playtime_2weeks  
0             Counter-Strike               6.0              0.0  
1      Team Fortress Classic               0.0              0.0  
2              Day of Defeat               7.0              0.0  
3         Deat

In [ ]:
df2_items

***Se revisan tipos de datos y se cambian si es necesario**

In [37]:

print(df2_items.dtypes)
df_items['item_id'] = df_items['item_id'].astype(str)


user_id              object
items_count           int64
steam_id             object
user_url             object
item_id              object
item_name            object
playtime_forever    float64
playtime_2weeks     float64
dtype: object


Se exporta para el sistema de recomendación

In [39]:
df2_items.to_parquet('df2_items.parquet', engine='pyarrow', index=False)

**Se escogen las columnas que se necesitan para realizar las funciones de la API**

In [40]:

columnas_especificas2 = ['user_id','item_id','items_count','playtime_forever']

# Crear un nuevo DataFrame con solo las columnas específicas
nuevo_df_items = df2[columnas_especificas2]

# Ver el nuevo DataFrame
print(nuevo_df_items)
df_items = nuevo_df_items

                   user_id item_id  items_count  playtime_forever
0        76561197970982479      10          277               6.0
1        76561197970982479      20          277               0.0
2        76561197970982479      30          277               7.0
3        76561197970982479      40          277               0.0
4        76561197970982479      50          277               0.0
...                    ...     ...          ...               ...
5170010  76561198329548331  373330            7               0.0
5170011  76561198329548331  388490            7               3.0
5170012  76561198329548331  521570            7               4.0
5170013  76561198329548331  519140            7               3.0
5170014  edward_tremethick     NaN            0               NaN

[5170015 rows x 4 columns]


In [41]:
df_items = df_items.dropna(subset=['user_id','item_id','items_count','playtime_forever'])

In [43]:
df_items.shape

(5153209, 4)

**Se exporta para la API**

In [139]:
df_items.to_parquet('df_items.parquet', engine='pyarrow', index=False)